In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image
import cv2


class Dataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, files, ppm, tranform):
        self.filenames = files
        self.list_IDs = list_IDs
        self.ppm = ppm
        self.transform = tranform

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        ID = self.list_IDs[index]
        cv_in = cv2.imread('../img/'+self.filenames[ID]+'.jpg',1) # color
        pil_in = Image.fromarray(cv_in)
        X = self.transform(pil_in)
        y = self.ppm[ID]
        return X,y

def double_conv(in_channels, out_channels):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True),nn.Conv2d(out_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True))

class LeUNet(nn.Module):

    def __init__(self):
        super(LeUNet,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = nn.Sequential(nn.Conv2d(1, 6, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),nn.Conv2d(6, 16, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),)
        self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        self.estimator = nn.Sequential(nn.Linear(16*5*5,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = self.features(x)
        x = self.avgpool(x)
        return x

In [2]:
def evaluate(loader,model,criterion):
    model.eval()
    loss = []
    pred = []
    actual = []
    #with torch.no_grad():
    for x, y in loader:
        y = y.float()
        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)
        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        loss.append(loss.data[0])
        print(y)
        print(yhat.squeeze())
        #actual.append(y[0]) #fix: print
        #pred.append(yhat.squeeze()[0]) #print
        
    return pred,actual,loss


data = pd.read_csv('../final_data.csv')
files = list(data['filename'])
ppm = list(data['ppm'])
ids = [i for i in range(len(files))]
#data = None

model = LeUNet()
model = torch.nn.DataParallel(model) #.cuda()
model.load_state_dict(torch.load("model_pm_best.pth"))
criterion = nn.MSELoss() #.cuda()
dataset = Dataset(ids, files, ppm, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])) # normalize
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

pred,actual,loss = evaluate(loader,model,criterion) # plot preds vs actual
# eval bad vs good examples from pred, actual (index numbered 0,1,2,...)


NameError: name 'LeUNet' is not defined